In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [3]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/data set/Data set/training'
valid_path = '/content/drive/MyDrive/data set/Data set/testing'

In [4]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87910968/87910968 [==============================] - 2s 0us/step


In [5]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [6]:
  # useful for getting number of output classes
folders = glob('/content/drive/MyDrive/data set/Data set/training/*')

In [7]:
folders

['/content/drive/MyDrive/data set/Data set/training/dio',
 '/content/drive/MyDrive/data set/Data set/training/ct100']

In [8]:
# our layers - you can add more if you want
x = Flatten()(inception.output)
prediction = Dense(len(folders), activation='softmax')(x)

In [9]:
# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [11]:

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [12]:

# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [13]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/data set/Data set/training',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 2845 images belonging to 2 classes.


In [14]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/data set/Data set/testing',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 706 images belonging to 2 classes.


In [15]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-15-5932e3024fde>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/10
89/89 [==============================] - 820s 9s/step - loss: 0.6992 - accuracy: 0.9378 - val_loss: 0.2378 - val_accuracy: 0.9674
Epoch 2/10
89/89 [==============================] - 523s 6s/step - loss: 0.3410 - accuracy: 0.9631 - val_loss: 0.4076 - val_accuracy: 0.9688
Epoch 3/10
89/89 [==============================] - 524s 6s/step - loss: 0.2130 - accuracy: 0.9810 - val_loss: 0.5304 - val_accuracy: 0.9632
Epoch 4/10
89/89 [==============================] - 524s 6s/step - loss: 0.1789 - accuracy: 0.9831 - val_loss: 0.2191 - val_accuracy: 0.9759
Epoch 5/10
89/89 [==============================] - 520s 6s/step - loss: 0.1248 - accuracy: 0.9852 - val_loss: 0.3268 - val_accuracy: 0.9703
Epoch 6/10
89/89 [==============================] - 521s 6s/step - loss: 0.1696 - accuracy: 0.9838 - val_loss: 0.2191 - val_accuracy: 0.9788
Epoch 7/10
89/89 [==============================] - 464s 5s/step - loss: 0.1149 - accuracy: 0.9905 - val_loss: 0.1953 - val_accuracy: 0.9802
Epoch 8/10
89